In [6]:
import pyodbc
import pandas as pd
import spacy
import nltk
from nltk.probability import FreqDist
import regex as re

In [7]:

conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server= TPCCP-DB09\SCNEAR;'
                      'Database=Hughes;'
                      'Trusted_Connection=yes;')


data ='''SELECT [Ccmsid]
      ,[MonitorDate]
      ,[CommentCaseId]
      ,[CommentComentarios]
      ,[KpiAEvaluar]
      ,[Pais]
      ,[RcaLevelOne]
      ,[RcaLevelTwo]
      ,[RcaLevelThree]
      ,[RcaLevelFour]
      ,[RcaLevelFive]
      ,[TopContactDriver]
  FROM [Hughes].[dbo].[tbFormHughes]'''

df = pd.read_sql(sql=data,con=conn)
df.to_csv('output/detractor_analyisis.csv', index=False)


In [27]:




data ='''SELECT  [EndDate]
      ,[CaseID]
      ,[CSAT]
      ,[DSAT]
      ,[IR]
      ,[FCR]
      ,[NEUTRAL]
      ,[CcmsId]
      ,[MonthNum]
      ,[Month]
      ,[Year]
      ,[Week(Year)]
      ,[AgentName]
      ,[Supv]
      ,[Wave]
      ,[SurveyTenure]
      ,[Days]
      ,[Courtesy]
      ,[Knowledgeable]
      ,[Easy To Follow]
      ,[Authority]
      ,[Understandability]
      ,[Tenure]
      ,[Country]
      ,[ProblemTypeDetail]
      ,[PorfavorDanosCualquierComentarioAdicional]
  FROM [Hughes].[dbo].[tbTPCOCSAT]'''
df = pd.read_sql(sql=data,con=conn)

df.rename(columns={'EndDate': 'Date', 'CcmsId':'CCMS', 'Days':'Tenure Time','Supv':'Supervisor',
                             'AgentName':'Agent', 'ProblemTypeDetail':'Contact Driver', 'PorfavorDanosCualquierComentarioAdicional':'Feedback'}, inplace=True)

df['Date']=pd.to_datetime(df['Date']).dt.date

df['Contact Driver 2']=df['Contact Driver'].apply(lambda x: x.split('-')[1])
df['Contact Driver']=df['Contact Driver'].apply(lambda x: x.split('-')[0])


def categorize_response(row):
    if row['CSAT'] == 1:
        return 'Promoter'
    elif row['NEUTRAL'] == 1:
        return 'Neutral'
    elif row['DSAT'] == 1:
        return 'Detractor'
    else:
        return 'Unknown'


df['CSAT Response'] = df.apply(categorize_response, axis=1)


df['IR Response']=df['IR'].apply(lambda x: 'Yes' if x is 1 else 'No')
df['FCR Response']=df['FCR'].apply(lambda x: 'Yes' if x is 1 else 'No')
df.to_csv('output/surveys_raw.csv', index=False)
df_surveys_raw=df.copy
df.head()

,Date,CaseID,CSAT,DSAT,IR,FCR,NEUTRAL,CCMS,MonthNum,Month,...,Authority,Understandability,Tenure,Country,Contact Driver,Feedback,Contact Driver 2,CSAT Response,IR Response,FCR Response
0,2023-01-02,6730368,1,0,1,1,0,4966098,1,January,...,1,0,Operation,CO,Billing Info,El agente fue amable y resolvio mi problema.,Explanation of Charges,Promoter,Yes,Yes
1,2023-01-02,6753433,1,0,1,1,0,4952997,1,January,...,1,0,Operation,CO,Reactivate,Lo uniko esque las cuotas son muy altas pregun...,Reactivate from Suspension,Promoter,Yes,Yes
2,2023-01-02,6753616,1,0,1,1,0,4931540,1,January,...,1,0,Operation,CO,Billing Info,,Explanation of Charges,Promoter,Yes,Yes
3,2023-01-02,6753654,0,1,1,1,0,4983763,1,January,...,0,0,Nesting,PE,Slow Internet Connectivity,Facilidad de contacto mas rÃ¡pido llamadas por...,Exceeded Data Allowance,Detractor,Yes,Yes
4,2023-01-02,6754670,0,1,0,0,0,4967960,1,January,...,0,0,Operation,CO,No internet connectivity,Llevo dos meses sin servicio solicito reembols...,Hughes Outage/Degradation,Detractor,No,No


In [9]:

df

,Date,CaseID,CSAT,DSAT,IR,FCR,NEUTRAL,CCMS,MonthNum,Month,...,Courtesy,Knowledgeable,Easy To Follow,Authority,Understandability,Tenure,Country,Contact Driver,Feedback,Contact Driver 2
0,2023-01-02,6730368,1,0,1,1,0,4966098,1,January,...,1,1,1,1,0,Operation,CO,Billing Info,El agente fue amable y resolvio mi problema.,Explanation of Charges
1,2023-01-02,6753433,1,0,1,1,0,4952997,1,January,...,1,1,1,1,0,Operation,CO,Reactivate,Lo uniko esque las cuotas son muy altas pregun...,Reactivate from Suspension
2,2023-01-02,6753616,1,0,1,1,0,4931540,1,January,...,1,1,1,1,0,Operation,CO,Billing Info,,Explanation of Charges
3,2023-01-02,6753654,0,1,1,1,0,4983763,1,January,...,0,0,0,0,0,Nesting,PE,Slow Internet Connectivity,Facilidad de contacto mas rÃ¡pido llamadas por...,Exceeded Data Allowance
4,2023-01-02,6754670,0,1,0,0,0,4967960,1,January,...,0,0,0,0,0,Operation,CO,No internet connectivity,Llevo dos meses sin servicio solicito reembols...,Hughes Outage/Degradation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2125,2023-06-05,7311923,0,0,0,0,1,3134957,6,June,...,1,0,0,0,0,Operation,CO,No internet connectivity,NECESITO URGENTEMENTE U NA ORDEN DE SERVICIO P...,Customer device issue
2126,2023-06-05,7312899,1,0,1,1,0,3134957,6,June,...,1,1,1,1,0,Operation,CO,Billing Info,La señorita Diana Londoño muy atenta y servici...,Explanation of Charges
2127,2023-06-05,7312973,1,0,1,1,0,4941777,6,June,...,1,1,1,1,0,Operation,CL,Billing Info,"Cuando use el servicio lo encontré deficiente,...",Payment Voucher
2128,2023-06-05,7313021,1,0,1,1,0,5025248,6,June,...,1,1,1,1,0,Operation,CL,Slow Internet Connectivity,Bueno la cobertura es lenta no se puede comuni...,Bad Modem


In [10]:
df = df[['CaseID', 'Feedback']]
print("Number of surveys",len(df))


df=df[df['Feedback'].notnull()]
print("After filtering null",len(df))


df_sentiment = df.copy()

Number of surveys 2130
After filtering null 2128


In [11]:
nlp_es = spacy.load('es_core_news_md')

In [12]:
df['Feedback']=df['Feedback'].apply(lambda x: str(x).lower())
#Remove special characters

special_chars1 = df['Feedback'].apply(lambda x: [each for each in list(x) if not each.isalnum() and each != ' '] if type(x) == type('Hello') else '')
flat_list1 = [item for sublist in special_chars1 for item in sublist]
print(set(flat_list1)) # Set = List of character with no repeat
special_char_to_remove = set(flat_list1)

def remove_special_char(words):
    return [word for word in words if not str(word) in special_char_to_remove]

df['spacy']=df['Feedback'].apply(lambda x: remove_special_char(nlp_es(str(x))))

df['Lemmatization'] = df['spacy'].apply(lambda doc: [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.is_space and len(token) > 2 and token.is_alpha]) # doc if str(doc) == 'nan' else # not token.is_stop and



df_cat = df.copy()
df_cat['joint_lemma_words'] = df_cat['Lemmatization'].apply(lambda x: ' '.join(x))
df_cat.drop(['Lemmatization', 'Feedback','spacy'], axis = 1, inplace = True)

df.to_csv("output/freqwords.csv",index=False, sep=';', encoding='utf-8')

df_freq_words=df.copy()
df.head()

{'¡', '\n', ';', ':', '-', '±', '!', ')', '~', '.', '(', ',', '@', '¨', '…', '%', '/', '\xad', '?'}


,CaseID,Feedback,spacy,Lemmatization
0,6730368,el agente fue amable y resolvio mi problema.,"[el, agente, fue, amable, y, resolvio, mi, pro...","[agente, amable, resolver, problema]"
1,6753433,lo uniko esque las cuotas son muy altas pregun...,"[lo, uniko, esque, las, cuotas, son, muy, alta...","[uniko, escar, cuota, alto, pregunto]"
2,6753616,,[],[]
3,6753654,facilidad de contacto mas rã¡pido llamadas por...,"[facilidad, de, contacto, mas, rã¡pido, llamad...","[facilidad, contacto, llamado, whatsapp, resol..."
4,6754670,llevo dos meses sin servicio solicito reembols...,"[llevo, dos, meses, sin, servicio, solicito, r...","[llevar, mes, servicio, solicito, reembolso, s..."


In [13]:
# Drop 'Feedback' and 'spacy' columns
df.drop(['Feedback', 'spacy'], axis=1, inplace=True)

#Super Important! 
df.reset_index(drop=True, inplace=True)

df_bigram=df.copy()
df_trigram=df.copy()

df.head()

,CaseID,Lemmatization
0,6730368,"[agente, amable, resolver, problema]"
1,6753433,"[uniko, escar, cuota, alto, pregunto]"
2,6753616,[]
3,6753654,"[facilidad, contacto, llamado, whatsapp, resol..."
4,6754670,"[llevar, mes, servicio, solicito, reembolso, s..."


In [14]:
df["frecwords"]=df['Lemmatization'].apply(lambda x: FreqDist(x))
df=df.drop(['Lemmatization'], axis = 1)

df_temp = pd.DataFrame(pd.DataFrame(df["frecwords"].values.tolist()).stack().reset_index(level=1))
df_temp.columns = ['Words', 'Frequency']
df= df.join(df_temp)

df.drop(["frecwords"], axis = 1, inplace = True)

# Removing unnecesary blankspaces around words
df['Words'] = df['Words'].apply(lambda x: str(x).strip())

df.dropna(inplace=True)
df['Frequency'] = df['Frequency'].astype('int8')
df.to_csv(r"output/Words.csv", encoding = 'utf-8', sep=';', index=False)

df.head()

,CaseID,Words,Frequency
0,6730368,agente,1
0,6730368,amable,1
0,6730368,resolver,1
0,6730368,problema,1
1,6753433,uniko,1


In [15]:

## Bigrams 

def common_bigrams(words_list):
    list_bigrams = list(nltk.bigrams(words_list))
    filtered_bigram_dist = FreqDist(list_bigrams)
    return dict(filtered_bigram_dist)

df_bigram['bigrams_dict'] = df_bigram['Lemmatization'].apply(lambda x: common_bigrams(x))

# Melt frequency distribution dictionary
# https://stackoverflow.com/questions/61354434/melt-pandas-dataframe-containing-column-of-dictionaries-such-that-the-dictionary
df_temp_bigrams = pd.DataFrame(pd.DataFrame(df_bigram['bigrams_dict'].values.tolist()).stack().reset_index(level=1))
df_temp_bigrams.columns = ['Bigrams_tuple', 'bigrams_count']

display(df_temp_bigrams)
# Split bigram tuple in columns
# https://stackoverflow.com/questions/29550414/how-can-i-split-a-column-of-tuples-in-a-pandas-dataframe
df_temp_bigrams[['bigram_1', 'bigram_2']] = pd.DataFrame(df_temp_bigrams['Bigrams_tuple'].tolist(), index=df_temp_bigrams.index)

df_bigram = df_bigram.join(df_temp_bigrams)

df_bigram.drop(['Lemmatization', 'bigrams_dict','Bigrams_tuple' ], axis = 1, inplace = True)
df_bigram['Bigrams'] = df_bigram['bigram_1'] +" " +df_bigram['bigram_2']
df_bigram.dropna(subset=['bigrams_count'], inplace=True)
df_bigram['bigrams_count'] = df_bigram['bigrams_count'].astype('int8')
df_bigram.to_csv("output/Bigrams.csv", sep=';', encoding='utf-8', index=False)
df_bigram.head()

,Bigrams_tuple,bigrams_count
0,"(agente, amable)",1.0
0,"(amable, resolver)",1.0
0,"(resolver, problema)",1.0
1,"(uniko, escar)",1.0
1,"(escar, cuota)",1.0
...,...,...
2126,"(teléfono, celular)",1.0
2127,"(pagar, servicio)",1.0
2127,"(económico, pagar)",1.0
2127,"(valor, económico)",1.0


,CaseID,bigrams_count,bigram_1,bigram_2,Bigrams
0,6730368,1,agente,amable,agente amable
0,6730368,1,amable,resolver,amable resolver
0,6730368,1,resolver,problema,resolver problema
1,6753433,1,uniko,escar,uniko escar
1,6753433,1,escar,cuota,escar cuota


In [16]:
#Trigrams

def common_trigrams(words_list):
    list_trigrams = list(nltk.trigrams(words_list))
    filtered_trigrams_dist = FreqDist(list_trigrams)
    return dict(filtered_trigrams_dist)


df_trigram['trigrams_dict'] = df_trigram['Lemmatization'].apply(lambda x: common_trigrams(x))

# Melt frequency distribution dictionary
# https://stackoverflow.com/questions/61354434/melt-pandas-dataframe-containing-column-of-dictionaries-such-that-the-dictionary
df_temp_trigrams = pd.DataFrame(pd.DataFrame(df_trigram['trigrams_dict'].values.tolist()).stack().reset_index(level=1))
df_temp_trigrams.columns = ['Trigrams_tuple', 'Trigrams_count']

display(df_temp_trigrams)

df_temp_trigrams[['trigram_1', 'trigram_2', 'trigram_3']] = pd.DataFrame(df_temp_trigrams['Trigrams_tuple'].tolist(), index=df_temp_trigrams.index)

df_trigram = df_trigram.join(df_temp_trigrams)

df_trigram.drop(['Lemmatization', 'trigrams_dict','Trigrams_tuple' ], axis = 1, inplace = True)
df_trigram['Trigrams'] = df_trigram['trigram_1'] +" " +df_trigram['trigram_2'] +" " +df_trigram['trigram_3']
df_trigram.dropna(subset=['Trigrams_count'], inplace=True)
df_trigram['Trigrams_count'] = df_trigram['Trigrams_count'].astype('int8')
df_trigram.to_csv("output/Trigrams.csv", sep=';', encoding='utf-8', index=False)
df_trigram.head()

,Trigrams_tuple,Trigrams_count
0,"(agente, amable, resolver)",1.0
0,"(amable, resolver, problema)",1.0
1,"(uniko, escar, cuota)",1.0
1,"(escar, cuota, alto)",1.0
1,"(cuota, alto, pregunto)",1.0
...,...,...
2126,"(lento, comunicar, teléfono)",1.0
2126,"(comunicar, teléfono, celular)",1.0
2127,"(económico, pagar, servicio)",1.0
2127,"(valor, económico, pagar)",1.0


,CaseID,Trigrams_count,trigram_1,trigram_2,trigram_3,Trigrams
0,6730368,1,agente,amable,resolver,agente amable resolver
0,6730368,1,amable,resolver,problema,amable resolver problema
1,6753433,1,uniko,escar,cuota,uniko escar cuota
1,6753433,1,escar,cuota,alto,escar cuota alto
1,6753433,1,cuota,alto,pregunto,cuota alto pregunto


In [17]:
from tqdm.auto import tqdm, trange
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

In [18]:
# Sentiment 


######################### Model settings ############################
MODEL = f"finiteautomata/beto-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
######################################################################

######################### Use model #################################
def get_sentiment_scores(tweet):
    encoded_input = tokenizer(tweet, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    return scores
######################################################################

########################################### Get sentiment #######################################################
tokenizer.model_max_length = 1028
lst_negative_scores = []
lst_neutral_scores = []
lst_positive_scores = []
lst_label = []

df_sentiment['Feedback'] = df_sentiment['Feedback'].astype('str')
df_sentiment=df_sentiment[df_sentiment['Feedback'].str.len() < 1500]

for i in trange(df_sentiment.shape[0]):
    tweet_i = str(df_sentiment['Feedback'].values[i])
    
    scores_array = get_sentiment_scores(tweet_i)
    negative_score, neutral_score, positive_score = scores_array
    lst_negative_scores.append(negative_score)
    lst_neutral_scores.append(neutral_score)
    lst_positive_scores.append(positive_score)
    
    ranking = np.argsort(scores_array)
    ranking = ranking[::-1]
    label = config.id2label[ranking[0]]
    lst_label.append(label)
    
df_sentiment['NEGATIVE_SCORE'] = lst_negative_scores
df_sentiment['NEUTRAL_SCORE'] = lst_neutral_scores
df_sentiment['POSITIVE_SCORE'] = lst_positive_scores
df_sentiment['LABEL'] = lst_label



df_sentiment['NEGATIVE_SCORE'] = df_sentiment['NEGATIVE_SCORE'].astype('str')
df_sentiment['NEGATIVE_SCORE'] = df_sentiment['NEGATIVE_SCORE'].apply(lambda x: x.replace('.',','))

df_sentiment['NEUTRAL_SCORE'] = df_sentiment['NEUTRAL_SCORE'].astype('str')
df_sentiment['NEUTRAL_SCORE'] = df_sentiment['NEUTRAL_SCORE'].apply(lambda x: x.replace('.',','))

df_sentiment['POSITIVE_SCORE']=df_sentiment['POSITIVE_SCORE'].astype('str')
df_sentiment['POSITIVE_SCORE'] = df_sentiment['POSITIVE_SCORE'].apply(lambda x: x.replace('.',','))

df_sentiment.drop(['Feedback'], axis = 1, inplace = True)

df_sentiment.to_csv('output/Sentiment.csv', sep=';', index=False)
df_sentiment

  0%|          | 0/2128 [00:00<?, ?it/s]

,CaseID,NEGATIVE_SCORE,NEUTRAL_SCORE,POSITIVE_SCORE,LABEL
0,6730368,"0,0006746173","0,08517327","0,914152",POS
1,6753433,"0,5486768","0,4499489","0,0013742076",NEG
2,6753616,"0,027255414","0,89861774","0,07412671",NEU
3,6753654,"0,0007724044","0,9532539","0,0459737",NEU
4,6754670,"0,16149755","0,83663577","0,0018667186",NEU
...,...,...,...,...,...
2125,7311923,"0,8880284","0,05977322","0,052198436",NEG
2126,7312899,"0,00041302253","0,0014790059","0,99810773",POS
2127,7312973,"0,0028121308","0,9964297","0,0007583668",NEU
2128,7313021,"0,94637877","0,052840903","0,0007802462",NEG


In [19]:
### Categorization

def remove_nonalpha(words):
    list_nonalpha = [w.strip() for w in words if w.strip().isalpha()]        
    return list_nonalpha 
    
def get_category(x):
    key_list = []
    match_list = []
    for key in categories_dict.keys():
        for value in categories_dict[key]:
            if (bool(re.search(value,x)) ==True):
                if key not in key_list:
                    key_list.append(key)
                    match_list.append(re.compile(value).findall(x))
    return dict(zip(key_list, match_list))

df_agent = df_cat.copy()
df_channel=df_cat.copy()

In [20]:
categories_dict = {
'Precio' : [
'(costoso|precio)(\s|\w+\s|\s\w+\s){0,3}(servicio|internet|plan)',
'(servicio|internet|plan)(\s|\w+\s|\s\w+\s){0,3}(costoso)',
'(mejorar)(\s|\w+\s|\s\w+\s){0,3}(costo)',
'(caro)',
'(costo)',
'(tarifa)(\s|\w+\s|\s\w+\s){0,3}(elevado|alto)',
'(bajar|rebajar|rebaja)(\s|\w+\s|\s\w+\s){0,3}(tarifa|mensualidad)'
'(descuento)(\s|\w+\s|\s\w+\s){0,3}(factura|cliente|antiguo|navideño)',
'(ofrecer|dejar|fijo|beneficio|hacer|pedir|brindarme|acceder|solicito)(\s|\w+\s|\s\w+\s){0,3}(descuento)'
],

'Calidad del servicio':[
'lento',
'velocidad',
'(internet|servicio)(\s|\w+\s|\s\w+\s){0,3}(caer|falla|técnico|malo)',
'(activen|señal|conexion)(\s|\w+\s|\s\w+\s){0,3}(internet)',
'(compania)(\s|\w+\s|\s\w+\s){0,3}(peorcito)',
'(error)(\s|\w+\s|\s\w+\s){0,3}(prestir)',
'(resolver)(\s|\w+\s|\s\w+\s){0,3}(atención)',
'(soporte)(\s|\w+\s|\s\w+\s){0,3}(técnico)'
]
,
'Tiempo':[
'(edperar|esperar|demorar|seguimiento)(\s|\w+\s|\s\w+\s){0,3}(tiempo|contestir|contestar|instalación)',
'(cliente|atender)(\s|\w+\s|\s\w+\s){0,3}(esperar)',
'(resuelir|reclamo|realizar)(\s|\w+\s|\s\w+\s){0,3}(puntualidad)',
'(resolver|problema|ayuda|recibirla|esperar|resuelir|corto|edperar|pronto|compensar|demorar)(\s|\w+\s|\s\w+\s){0,3}(tiempo|servicio)',
'(resolucion|problema)(\s|\w+\s|\s\w+\s){0,3}(engorroso)'
        ],

'Cancelar':[
' cancelar'
],

'Instalación':[
'(mantenimiento|cambiar|revisar|problema)(\s|\w+\s|\s\w+\s){0,3}(antena|router)',
'(antena|router)(\s|\w+\s|\s\w+\s)(mantenimiento|cambiar|revisar){0,3}',
'(contar)(\s|\w+\s|\s\w+\s){0,3}(internet)',
'(internet)(\s|\w+\s|\s\w+\s){0,3}(pagado|acceder)',
'(instalación)(\s|\w+\s|\s\w+\s){0,3}(error)'
],


'Facturación':[
    '(error)(\s|\w+\s|\s\w+\s){0,3}(facturación)',
'(solucionar|tema)(\s|\w+\s|\s\w+\s){0,3}(pago)',
'(solucionar|resuelir|problema)(\s|\w+\s|\s\w+\s){0,3}(dinero)']

}






In [21]:
df_channel['cat_agent_dict'] = df_channel["joint_lemma_words"].apply(lambda x: get_category(x))

df_channel["dict_len"] = df_channel["cat_agent_dict"].apply(lambda x: len(x))
df_nohits2= df_channel[df_channel["dict_len"] == 0]
df_channel = df_channel[df_channel["dict_len"] > 0]
df_channel.reset_index(inplace = True, drop=True)

# Melt frequency distribution dictionary
# https://stackoverflow.com/questions/61354434/melt-pandas-dataframe-containing-column-of-dictionaries-such-that-the-dictionary
#df_temp_cat_agent = 
df_channel_temp = pd.DataFrame(pd.DataFrame(df_channel['cat_agent_dict'].values.tolist()).stack().reset_index(level=1))
#df_temp_bigrams.columns = ['Bigrams', 'bigrams_count']

df_channel_temp.columns = ['category', 'hit']
#df_temp_bigrams

df_channel_temp['hit'] = df_channel_temp['hit'].apply(lambda x: remove_nonalpha(list(x[0])))

# Split trigram tuple in columns
# https://stackoverflow.com/questions/29550414/how-can-i-split-a-column-of-tuples-in-a-pandas-dataframe
df_channel_temp = pd.concat([df_channel_temp, pd.DataFrame(df_channel_temp['hit'].tolist(), index=df_channel_temp.index)], axis=1)
df_channel_temp['hit'] = df_channel_temp['hit'].apply(lambda x : ' '.join(x))

df_channel = df_channel.join(df_channel_temp)

df_channel.drop(['cat_agent_dict', 'dict_len'], axis = 1, inplace = True)
df_channel['categorization_level'] = 'Brand'
df_channel.head()

,CaseID,joint_lemma_words,category,hit,0,1,2,3,4,5,6,7,8,categorization_level
0,6758753,servicio costoso,Precio,servicio costoso,servicio,costoso,None,None,None,None,None,None,None,Brand
1,6739733,caro servicio,Precio,c a r o,c,a,r,o,None,None,None,None,None,Brand
2,6755293,rapida atencion gustariar capacidad velocidad ...,Calidad del servicio,v e l o c i d a d,v,e,l,o,c,i,d,a,d,Brand
3,6772582,esperar reconexión servicio demorar,Tiempo,esperar reconexión servicio,esperar,reconexión,servicio,None,None,None,None,None,None,Brand
4,6769240,servicio caro trabajo remoto,Precio,c a r o,c,a,r,o,None,None,None,None,None,Brand


In [22]:
categories_dict = {
'Irrespetuoso':[
    '(grosero|grosera)'
],

'Amabilidad':[
    '(amable|paciente|ameno|cálido)'
],
'Espera Larga':[
    '(dejar)(\s|\w+\s|\s\w+\s){0,3}(espera)',
    '(colgar|colgoir)(\s|\w+\s|\s\w+\s){0,3}(llamada)'
    '(tiempo)(\s|\w+\s|\s\w+\s){0,3}(atendar)'],

'Conocimiento':[
    '(conociar|conocer)(\s|\w+\s|\s\w+\s){0,3}(tema)',
    '(orientar|operador)(\s|\w+\s|\s\w+\s){0,3}(saber)',
    '(explicar)']
}

In [23]:
df_agent['cat_agent_dict'] = df_agent["joint_lemma_words"].apply(lambda x:get_category(x))

df_agent["dict_len"] = df_agent["cat_agent_dict"].apply(lambda x: len(x))
df_nohits1= df_agent[df_agent["dict_len"] == 0]
df_agent = df_agent[df_agent["dict_len"] > 0]

df_agent.reset_index(inplace = True, drop=True)

# Melt frequency distribution dictionary
# https://stackoverflow.com/questions/61354434/melt-pandas-dataframe-containing-column-of-dictionaries-such-that-the-dictionary
#df_temp_cat_agent = 
df_agent_temp = pd.DataFrame(pd.DataFrame(df_agent['cat_agent_dict'].values.tolist()).stack().reset_index(level=1))
#df_temp_bigrams.columns = ['Bigrams', 'bigrams_count']

df_agent_temp.columns = ['category', 'hit']
#df_temp_bigrams

df_agent_temp['hit'] = df_agent_temp['hit'].apply(lambda x: remove_nonalpha(list(x[0])))

# Split trigram tuple in columns
# https://stackoverflow.com/questions/29550414/how-can-i-split-a-column-of-tuples-in-a-pandas-dataframe
df_agent_temp = pd.concat([df_agent_temp, pd.DataFrame(df_agent_temp['hit'].tolist(), index=df_agent_temp.index)], axis=1)
df_agent_temp['hit'] = df_agent_temp['hit'].apply(lambda x : ' '.join(x))

df_agent = df_agent.join(df_agent_temp)

df_agent.drop(['cat_agent_dict', 'dict_len'], axis = 1, inplace = True)
df_agent['categorization_level'] = 'Agent'
df_agent.head()

,CaseID,joint_lemma_words,category,hit,0,1,2,3,4,5,6,7,categorization_level
0,6730368,agente amable resolver problema,Amabilidad,a m a b l e,a,m,a,b,l,e,None,None,Agent
1,6763078,explicar duda usuario segãºn problema avez cli...,Conocimiento,e x p l i c a r,e,x,p,l,i,c,a,r,Agent
2,6782194,agente encargado atender llamado amable,Amabilidad,a m a b l e,a,m,a,b,l,e,None,None,Agent
3,6784289,gracias ejecutiva srto estefania salazar dilig...,Amabilidad,a m a b l e,a,m,a,b,l,e,None,None,Agent
4,6792153,amable,Amabilidad,a m a b l e,a,m,a,b,l,e,None,None,Agent


In [24]:
df_cat = pd.concat([df_agent, df_channel])
df_cat=df_cat[['CaseID', 'category', 'categorization_level']]

df_cat = df_cat.merge(df_sentiment,on= 'CaseID', how='left')
df_cat.to_csv('output/Categorization.csv', index=False, sep=';')
df_cat.head()



NameError: name 'datetime' is not defined

In [25]:
from datetime import datetime
print('Last run', datetime.now().date())

Last run 2023-06-07
